# PRUEBA DE MODELOS - EMBEDDING Y CLASIFICACIÓN

Lo que se pretende en este libro de Jupyter es empezar con las pruebas de algunos modelos que se pudieron obtener de referencia en investigaciones anteriores. Todo esto con el fin de poder tener una mejor noción de estos modelos y en el mejor de los casos, ya tener un modelo definido con el que se trabajará posteriormente. 

Algunos de estos modelos son simplemente de embedding, otros tantos si conllevan algunas técnicas ya avanzadas de clasificación o detección de imágenes, según corresponda. La lista de modelos que se pretende probar es la siguiente:

- OpenL3 (Solo de embedding)

- ViT (de Google)

- Contrastors: Aquí podemos hacer el uso a su vez de dos modelos, CLIP y MRL

- Sports: Lista de varios modelos que nos pueden ayudar con la tarea asignada

- ResNet: Modelo moldeable con el número de capas que éste utiliza, es una red neuronal convolucional

- InceptionV3: Modelo moldeable con el número de capas y parámetros que éste utiliza, es una red neuronal convolucional

Ejemplos de modelos que algunas personas tomaron para la solución del proyecto. Tomado de [aquí](https://www.kaggle.com/competitions/planttraits2024/code)

1. Primer ejemplo: [KerasCV - EfficientNetV2](https://www.kaggle.com/code/awsaf49/planttraits2024-kerascv-starter-notebook#🔍-%7C-Loss-&-Metric)

Aquí en este ejemplo podemos ver los detalles del significado de cada _plant trait_ y qué significa cada uno. Aquí es importante definir algunos parámetros (que también he visto dentro de ViT, por ejemplo, y que son el tamño de las imágenes, epochs, seed, verbose, etc. Aquí también se hace el augmentations de las imágenes (flip, rotation, brightness, etc.), aquí los augmentations se hacen a un batch, lo cual acelera el entrenamiento y reduce el cuello de botella de CPU. 

Dentro del procedimiento de aplicar los augmenters, se ven funciones dentro de funciones, lo cual me confunde un poco. Después del proceso de augmenter, viene un proceso de la decodificación de las imágenes el cual involucra una modificación en el tamaño de la imagen y asignación de etiquetas (aquí hay más del proceso del mejoramiento de la imagen pero es un poco confuso)

Divide la data en 5 conjuntos, después crea segmentos basados en las 6 _plant traits_ y las combina dentro de una columna final de segmento, al final, utiliza este segmento para balancear distribuciones similares de segmentos a través de los conjuntos. 

Antes de entrenar el modelo como tal, se aplica una normalización estándar usando StandardScaler, esto asegura que los features tengan escalas consistentes, el cual es crucial para el desempeño óptimo de las capas lineales o densas --**Build Train & Valid Dataset**--

## Modelo 1: OpenL3

Las especificaciones de este modelo se pueden encontrar en la página de [GitHub](https://github.com/marl/openl3?tab=readme-ov-file) o su [Documentación](https://openl3.readthedocs.io/en/latest/tutorial.html#introduction) oficial. [API Reference](https://openl3.readthedocs.io/en/latest/api.html) 

Papers de referencia:

- [Paper 1: Look Listen and Learn](https://openaccess.thecvf.com/content_ICCV_2017/papers/Arandjelovic_Look_Listen_and_ICCV_2017_paper.pdf)

- [Paper 2: Look, Listen and Learn more: design choices for deep audio embeddings](http://www.justinsalamon.com/uploads/4/3/9/4/4394963/cramer_looklistenlearnmore_icassp_2019.pdf)


Los parámetros para los métodos de este modelo son los siguientes:

- content_type: "env", "music" (default). "music" es para videos, imágenes o música

- input_repr: "linear", "mel128" (default), "mel256"

- embedding_size: 512, 8192 (default). Tamaño del array resultante con el embedding de la imagen

Si el embedding ya existe, entonces no crea uno nuevo, deja el "original"

Para este modelo, existen 3 posibilidades:

1. Puedes ejecutar el modelo directamente a una imagen (o lista de imágenes) con el método "get_image_embedding"

2. Puedes guardar el embedding en la misma carpeta de donde viene la imagen para un uso posterior. Para guardar el embedding es el método "process_image_file" y para cargarlo es el método "np.load" con np la librería "numpy"

3. Puedes pre cargar desde un principio el modelo para que no estés cargandolo cada que lo requieras para una imagen. El método para pre cargar el modelo es "openl3.models.load_image_embedding_model", después, para usarlo en los métodos de los puntos anteriores, pasas el modelo con el argumento "model"

- Del método "imread" obtuvimos un array de matrices sobre la imagen

- De los métodos de openl3, los argumentos significan lo siguiente:

     1. input_repr: Representación del espectograma usado por el modelo. Es ignorado si el parámetro "modelo" es un modelo de tipo Keras válido. "linear-frequency log-magnitude spectogram", "Mel-frequency log-magnitude spectogram": este último captura información perceptivamente relevante de manera más eficiente con menos bandas de frecuencia (128 o 256 bandas) que el espectograma lineal

    2. content_type: Tipo de contenido utilizado para entrenar el modelo de embedding. Es ignorado si el parámetro "modelo" es un modelo de tipo Keras válido. "music" se refiere a contenido de música como tal, instrumentos, tonos, etc; "env" es de environmental y se refiere a sonidos humanos o de la naturaleza, aquellos que son reproducidos de manera "natural"

    3. embedding_size: Dimensión que tendrá el embedding. Es ignorado si el parámetro "modelo" es un modelo de tipo Keras válido

- Consideraciones extras sobre el modelo (obtenidas de los papers correspondientes):

    1. El espectrograma "Mel" captura información de manera más eficiente con menos bandas de frecuencia comparado con el espectrograma lineal, por eso siempre es mejor utilizar este argumento "Mel" a la hora de utilizar el modelo

    2. En la realización del modelo se utilizó el optimizador Adam para minimizar la pérdida de la entropia cruzada binaria con regularización L2
 
    3. Para las pruebas estadísticas del modelo se utilizó la Prueba de rangos de Wilcoxon
 
    4. El modelo L3 no requiere data con etiquetas, la representación Mel es la mejor dentro de este modelo
 
    5. El modelo L3 es no supervisado y aprende información a partir de entradas de de audio y visual al mismo tiempo
 
    6. Dentro de este modelo se utiliza la técnica de Activation Visualization el cual sirve para reconocer las regiones exactas de donde surgen los audios o videos, sus origenes.
 
- Al final, se pudo ejecutar el modelo de manera correcta y se tiene ya un mejor y mayor entendimiento del rendimiento y uso del modelo.

- **NOTA: Al ejecutar la función me sale un aviso de que estoy ejecutando una función costosa y me da los siguientes consejos para evitar estos cargos excesivos: Poner como argumento "reduce_tracing = True" o consultar documentación de TensorFlow [Doc1](https://www.tensorflow.org/guide/function#controlling_retracing) [Doc2](https://www.tensorflow.org/api_docs/python/tf/function)

In [11]:
#Librerias a utilizar en todo este proceso

#!pip install openl3

import openl3
from skimage.io import imread
import functions.general_functions as gf
import os

#Los embeddings se pueden leer con numpy
import numpy as np

ERROR: Could not find a version that satisfies the requirement tentensorflow<1.14 (from versions: none)
ERROR: No matching distribution found for tentensorflow<1.14


### 1.1 Aplicación directa del modelo

In [2]:
#Pre cargamos el modelo, al hacerlo solo una vez no es necesario pre cargar el modelo cada vez que se va a utilizar
modelo = openl3.models.load_image_embedding_model(input_repr="mel256", content_type="music", embedding_size=512)

#Variable global, de donde obtenemos la ruta de las imágenes de entrenamiento y prueba
ruta_imagenes_train = gf.get_data_path('train_images')
ruta_imagenes_test = gf.get_data_path('test_images')

In [3]:
#Método para generar los embeddings de manera directa a una sola imagen

#Nombre de la imagen a la cual aplicaremos el modelo
imagen_name = '993123.jpeg'

#Leemos la imagen
imagen1 = imread(ruta_imagenes_train + imagen_name)
#Generamos el embedding de la imagen de manera directa
emb = openl3.get_image_embedding(imagen1, content_type="env", input_repr="linear", embedding_size=512)

emb

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step


array([[ 1.175706  ,  1.5434446 ,  1.8470824 ,  1.6111743 ,  3.9059458 ,
         1.6161033 ,  1.1251589 ,  1.8484113 ,  1.2820483 ,  1.2422009 ,
         1.2345903 ,  0.83172244,  1.2606227 ,  1.6085217 ,  2.310515  ,
         1.9807837 ,  2.63912   ,  1.8400848 ,  1.7005421 ,  1.7075837 ,
         1.0354801 ,  2.1518202 ,  0.6044424 ,  1.4306686 ,  0.98116654,
         0.777962  ,  3.3654976 ,  4.162442  ,  1.9882624 ,  0.7811913 ,
         2.5927725 ,  1.8348336 ,  1.7911009 ,  1.8612864 ,  2.2643867 ,
         2.5106506 ,  1.129749  ,  0.7803635 ,  1.5808517 ,  2.0452437 ,
         0.7477303 ,  2.566805  ,  1.2202104 ,  2.673956  ,  1.3030437 ,
         0.9613706 ,  1.4589942 ,  1.1933473 ,  1.6517575 ,  1.4095986 ,
         1.3867158 ,  1.8570193 ,  3.5165267 ,  1.0719959 ,  0.7293594 ,
         2.3112679 ,  0.84064364,  2.1612198 ,  3.0060468 ,  1.9224309 ,
         1.1812272 ,  1.891209  ,  2.472405  ,  1.2888657 ,  1.6927787 ,
         2.1506999 ,  1.3459386 ,  2.0038981 ,  1.5

### 1.2 Guardar Embedding para uso futuro (Varisas imágenes)

In [4]:
#Aplicación del modelo a más de una imagen

#Rutas finales de las imágenes a procesar

imagen2 = ruta_imagenes_train + '998892.jpeg'
imagen3 = ruta_imagenes_train + '994535.jpeg'
#Lista para guardar todas las imágenes
imagen_array = [imagen2, imagen3]
#Método para guardar los embeddings de cada imagen en la misma carpeta de donde vienen las imágenes
openl3.process_image_file(imagen_array, batch_size = 32)

openl3: Processing /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/998892.jpeg (1/2)
openl3: /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/998892.npz exists, skipping.
openl3: Processing /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/994535.jpeg (2/2)
openl3: /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/994535.npz exists, skipping.


In [5]:
#En este método se leerán los embeddings de las imágenes generadas en la sección anterior

#Cargamos la data (embedding) de la imagen especificada
data = np.load(ruta_imagenes_train + '998892.npz')
#Obtenemos solo el embedding
emb = data['embedding']

emb[0]

array([-0.17171964,  2.7841794 ,  0.58611256, ...,  0.55718577,
        1.9009621 ,  1.2161709 ], dtype=float32)

In [6]:
#En esta sección se aplica el embedding a 10 imágenes distintas dentro de la carpeta de train images de manera directa

#Obtenemos la lista con los nombres (id's) de las imágenes de entrenamiento
train_file_names = os.listdir(ruta_imagenes_train)
#Nos quedamos solo con las primeras 10 imágenes
train_file_names = train_file_names[:10]

#Concatenamos el resto de la ruta del archivo al nombre de cada imagen
train_complete_file_names = [ruta_imagenes_train + x for x in train_file_names]

#Método para guardar los embeddings de cada imagen en la misma carpeta de donde vienen las imágenes
openl3.process_image_file(train_complete_file_names, batch_size = 32)

#Empezamos con el proceso de cargar los embeddings de cada imagen

#Lista en donde se guardará cada embedding
embs_files = list()
#Loop para ir guardando cada embedding
for imagen in train_file_names:
    #Cargamos la data (embedding) de la imagen especificada
    data = np.load(ruta_imagenes_train + imagen[:imagen.find('.')] + '.npz')
    #Obtenemos solo el embedding
    embs_files.append(data['embedding'][0])

embs_files[0]

openl3: Processing /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/174383279.jpeg (1/10)
openl3: /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/174383279.npz exists, skipping.
openl3: Processing /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/194747578.jpeg (2/10)
openl3: /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/194747578.npz exists, skipping.
openl3: Processing /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/196588153.jpeg (3/10)
openl3: /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/196588153.npz exists, skipping.
openl3: Processing /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/8324721.jpeg (4/10)
openl3: /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/8324721.npz exists, skipping.
openl3: Processing /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/179983287.jpeg (5/10)
openl3: /Users/pedrovela/Docs/Datasets - ML/planttrait

array([-0.47553322,  2.9182515 , -0.14319089, ...,  0.74044186,
        2.1243396 , -0.49356037], dtype=float32)

### 1.3 Uso del modelo pre cargado

In [7]:
#Método en el cual generamos el embedding de la imagen ya con el modelo pre cargado

#Obtenemos el embedding final
emb = openl3.get_image_embedding(imagen1, model=modelo)
emb

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step


array([[-0.20485121,  2.7495792 ,  1.1525512 ,  1.6354275 ,  1.5630054 ,
         3.0043402 ,  1.1680847 ,  0.20437273,  0.47765186,  1.6069542 ,
         0.99042135,  2.744746  ,  3.0113118 ,  0.697831  ,  2.5043488 ,
         2.8977034 ,  2.1103582 ,  1.0919865 ,  2.5187943 ,  2.6752896 ,
         2.5137546 ,  0.80452126,  0.9297262 ,  2.2770233 ,  2.8381968 ,
         0.6253177 ,  0.7834051 ,  1.436738  ,  1.1495699 ,  1.3403784 ,
         1.7564527 ,  1.0250019 ,  2.2259736 ,  0.5685906 ,  2.5769222 ,
         0.8931727 ,  2.3390589 ,  1.2697175 ,  1.2542069 ,  1.3876815 ,
         1.3700166 ,  1.7157243 ,  0.76253283,  1.8189112 ,  0.24554229,
         1.335274  ,  1.7735906 ,  1.3587192 ,  1.4913703 ,  1.041074  ,
         0.53341097,  0.9961289 ,  0.8008581 ,  1.6766714 ,  1.8453351 ,
         1.4003036 ,  1.7122384 ,  1.1727496 ,  1.851693  ,  2.0431597 ,
         2.2497199 ,  1.0162674 ,  2.1898563 ,  1.1334101 ,  0.64272827,
         1.9252778 ,  2.6810825 ,  1.1118598 ,  2.4

## Modelo 2: ViT (Google)

[GitHub](https://github.com/lukemelas/PyTorch-Pretrained-ViT?tab=readme-ov-file) , [Colab](https://colab.research.google.com/drive/1muZ4QFgVfwALgqmrfOkp7trAvqDemckO?usp=sharing)

Papers de referencia:

- [AN IMAGE IS WORTH 16X16 WORDS: TRANSFORMERS FOR IMAGE RECOGNITION AT SCALE](https://openreview.net/pdf?id=YicbFdNTTy)

- [MLP-Mixer: An all-MLP Architecture for Vision](https://arxiv.org/pdf/2105.01601)
 
- [How to train your ViT? Data, Augmentation, and Regularization in Vision Transformers](https://arxiv.org/pdf/2106.10270)
   
- [Attention Is All You Need](https://arxiv.org/pdf/1706.03762v7)

Proceso general del modelo:

1. Considera una imagen como una secuencia mono-dimensional (1 dimensión) de parches (token embeddings)
  
2. Antepone un token de clasificación a la secuencia generada en el punto 1

3. Pone estos parches a través de un codificador transformador (como BERT)

4. Pasa el primer token del output generado por el transformador a través de un pequeño MLP (Multilayer Perceptron) para obtener los logits de clasificación. Deja una capa escondida para pre-entrenamiento y una sola capa lineal para el fine-tunning

Puntos sobre la lectura:

- Se divide una imagen en parches y provee la secuencia de embeddings lineales de esos parches como entrada a un Transformador

- Los parches son tratados como Tokens (palabras) en una aplicación NLP

- El encoder Transformer consiste en alternar capas de multiheaded self-attention (MSA) y bloques MLP

- En este modelo existen dos tipos de capas: las capas MLP y MSA

- La longitud de la secuencia del Transformador es inversamente proporcional al cuadrado del tamaño del parche, por lo que modelos con tamaños pequeños de parches son computacionalmente más caros

- Aquí también se utiliza el optmizador Adam (la otra alternativa es SGD), es un optimizador estocástico. También existe el optmizador RAdam que brinda mejor control en la varianza del gradiente, el cual es necesario cuando el modelo se entrena a un valor alto de LR; RAdam detecta inestabilidad en la varianza y cambia el LR de manera suave para evitar la divergencia en las etapas iniciales del entrenamiento.

- **Adam (Adaptive Moment Estimation)**: Combina el optimizador RMSProp (Root Mean Square Propagation) con el Momentum. $\beta_1$ es el factor de olvido del gradiente y $\beta_2$ es el factor de olvido del segundo momento del gradiente

- **FLOPs (Floating Point Operations)**: Métricas que se utilizan comúnmente para calcular la complejidad computacional de los modelos de deep learning, sirve para entender fácilmente el número de operaciones aritméticas requeridas para realizar cierta operacion computacional. Número de operaciones de punto flotante (suma, resta, multiplicación y división en números de punto flotante). Al optimizar esta métrica podemos reducir la energía requerida para correr nuestra red neuronal [Ref 1](https://www.kdnuggets.com/2023/06/calculate-computational-efficiency-deep-learning-models-flops-macs.html) [Ref 2](https://medium.com/@pashashaik/a-guide-to-hand-calculating-flops-and-macs-fa5221ce5ccc)

- **EPOCH**: Se refiere al pase entero del conjunto de datos de entrenamientoa través de los algoritmos. Es un hiper-parametro que determina el proceso de entrenar el modelo entero de ML. Es cuando pasas TODO tu conjunto de entrenamiento a través del modelo y entonces se define como el número total de iteraciones que tuvo todo tu conjunto de entrenamiento para entrenar el modelo de ML, un "paso" de la data de entrenamiento se toma en cuenta cuando ésta haya pasado tanto por delante como por detrás en todos los procesos del modelo. Un epoch está completo cuando ha procesado todos los _batches_ y ha actualizado sus parámetros basados en el cálculo de pérdida. De manera general, incrementando el número de epochs mejora el performance del modelo al permitirle aprender comportamientos más complejos en la data, pero si existen muchos epochs, el modelo se sobre-entrena. Iteración completa: Procesar cada batch de la data de entrenamiento, calcular la pérdida y actualizar los parámetros del modelo. Ayuda a procesar o entrenar el modelo con un conjunto enorme de datos, si se tiene un conjunto enorme de datos y no es posible entrenar todos los datos de una sola vez (debido a memoria), los epochs te permiten entrenar el modelo en mini batches independientes que se realiza uno por uno. [Ref 1](https://www.simplilearn.com/tutorials/machine-learning-tutorial/what-is-epoch-in-machine-learning#:~:text=Machine%20learning%20models%20are%20trained,training%20data%20through%20the%20algorithm.) [Ref 2](https://www.geeksforgeeks.org/epoch-in-machine-learning/)

- **LEARNING RATE WARMUP**: Proceso para mejorar la optimización del modelo; incrementa gradualmente la taza (rate) de aprendizaje (tamaño del paso tomado en cada iteración para actualizar los parámetros del modelo) durante las etapas iniciales del entrenamiento antes de iniciar con las tazas de aprendizaje programadas; se incrementa linealmente la taza de aprendizaje durante las iteraciones iniciales y después se continua con las tazas programadas de aprendizaje. Se necesita ajustar los parámetros del calentamiento (warm-up) basado en características específicas del modelo y conjunto de datos. [Ref 1](https://techkluster.com/technology/learning-rate-warm-up/#:~:text=Learning%20rate%20warm%2Dup%20involves,how%20it%20can%20be%20implemented.) [Ref 2](https://www.baeldung.com/cs/learning-rate-warm-up)

- **SGD (Stochastic Gradient Descent)**: Método iterativo para optimizar la función objetivo con propiedades de suavidad adecuadas (diferenciabilidad, sub-diferenciabilidad), puede ser tomado como la aproximación estocástica de la técnica gradiente descendiente, ya que reemplaza el actual gradiente por otro gradiente estimado a partir de un subconjunto aleatorio de la data. Aquí, dentro del proceso de optimización de la función objetivo, en vez de calcular el verdadero gradiente de la función, lo que se hace es estimar ese gradiente con el gradiente de un solo registro o muestra, después actualiza este valor para cada entrada (muestra) del conjunto de entrenamiento hasta que se logra la conversión. Aquí se beneficia el costo computacional por iteración, aunque de manera general toma un número más alto de iteraciones que el método tradicional de gradiente descendiente. Este método es mejor para casos en donde el conjunto de datos es grande. Una elección de un valor alto de LR puede llevar a que el modelo tome pasos grandes en cada iteración y "pase" de largo el valor mínimo y una elección de LR baja puede ocasionar que el modelo converga lentamente. [Ref 1](https://en.wikipedia.org/wiki/Stochastic_gradient_descent) [Ref 2](https://www.geeksforgeeks.org/ml-stochastic-gradient-descent-sgd/) [Ref 3](https://towardsdatascience.com/stochastic-gradient-descent-clearly-explained-53d239905d31) [Ref 4](https://scikit-learn.org/stable/modules/sgd.html)

- fine-tunning con SGD con momentum de 0.9, el fine-tunning general del modelo se hace con una resolución de 384 (a menos que se estipule otra cosa). EL Vision Transformer se beneficia con mejores resoluciones.

- SGD con momentum recuerda la actualización del cambio en el parámetro w que minimiza la función objetivo en cada iteración y determina la sig. actualización del parámetro como una combinación lineal del gradiente y la actualización anterior. Utiliza un factor de decadencia entre 0 y 1 que determina la contribución relativa del actual gradiente y gradientes anteriores con respecto al cambio en los pesos. 

Rápida descripción de la inspección interna de ViT:

1. La primera capa de ViT proyecta linealmente los parches aplanados a un espacio con dimensión menor

2. Se agrega una posición de embedding a cada parche. El modelo aprende a calcular la distancia (similitud) entre las imagenes con respecto a su posición de embedding; parches cercanos tienden a tener posiciones de embeddings más similares. Esta distancia es de coseno entre la posición del embedding del parche de un determinado renglón y columna y la posición del embedding de los demás parches. Parches que se encuentran en la misma columna o renglón tienen embeddings similares

3. Una estructura sinusoidal (de tipo seno) se utiliza típicamente para mallas grandes

4. Self-attention permite a ViT integrar información a través de toda la imagen, incluso en las capas más bajas. Es una distancia que se mide entre el pixel de consulta (query) y los demás pixeles, todas con un peso de atención (weight). Mientras la capa sea más baja (profunda), la atención que se le da a la imagen es mayor.

5. El modelo atiende a regiones de la imagen que son semánticamente relevantes para la clasificación


**TRATAR DE REPLICAR ESTOS EJEMPLOS:** [Kaggle Example](https://www.kaggle.com/code/raufmomin/vision-transformer-vit-from-scratch) [Medium Example](https://medium.com/@diego.machado/fine-tuning-vit-for-image-classification-with-hugging-face-48c4be31e367) [Busqueda de Google (referencias)](https://www.google.com/search?q=using+vit+model+with+csv+file&client=safari&sca_esv=41a53a54b5bc3fc7&sca_upv=1&sxsrf=ADLYWIL7bNtclYzSlPXpYVpatUTD_zwpYw%3A1723677174417&source=hp&ei=9jm9Zry5F-7HkPIPiLy8iAo&iflsig=AL9hbdgAAAAAZr1IBha6iJ9JBpkWgf9h1nv91qK1KZIl&ved=0ahUKEwi8jL_tzfWHAxXuI0QIHQgeD6EQ4dUDCBY&uact=5&oq=using+vit+model+with+csv+file&gs_lp=Egdnd3Mtd2l6Ih11c2luZyB2aXQgbW9kZWwgd2l0aCBjc3YgZmlsZTIFECEYoAFIsj9QAFjZPnAAeACQAQCYAZ0BoAG_GKoBBDYuMjK4AQPIAQD4AQGYAhugArIYwgIKECMYgAQYJxiKBcICBBAjGCfCAg4QABiABBixAxiDARiKBcICERAuGIAEGLEDGNEDGIMBGMcBwgILEC4YgAQYsQMYgwHCAgsQABiABBixAxiDAcICBRAAGIAEwgIREC4YgAQYsQMYxwEYjgUYrwHCAggQLhiABBixA8ICCBAAGIAEGLEDwgIFEC4YgATCAg4QLhiABBjHARiOBRivAcICChAAGIAEGLEDGArCAgsQLhiABBjHARivAcICDRAAGIAEGLEDGEYY_wHCAgYQABgWGB7CAgcQABiABBgTwgIJEAAYgAQYExgKwgIIEAAYExgWGB7CAgQQIRgVmAMAkgcEMy4yNKAH734&sclient=gws-wiz)

El siguiente ejemplo se tomó de [GitHub](https://github.com/lukemelas/PyTorch-Pretrained-ViT?tab=readme-ov-file)

In [2]:
#Librerias a importar

#!pip3 install pytorch_pretrained_vit

from pytorch_pretrained_vit import ViT
model = ViT('B_16_imagenet1k', pretrained=True)

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.8/150.8 MB 29.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 20.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 4.5 MB/s eta 0:00:00
  Created wheel for pytorch_pretrained_vit: filename=pytorch_pretrained_vit-0.0.7-py3-none-any.whl size=11114 sha256=9f5a18f9e5c23c5849b1b1a6b48aaaf0e5b85aafac1e211526c97205ad4ea5e6
  Stored in directory: /Users/pedrovela/Library/Caches/pip/wheels/79/c8/4f/9ad72c6f247a7e6888cc7767db9632675cf82656fffec85518
Successfully built pytorch_pretrained_vit


Downloading: "https://github.com/lukemelas/PyTorch-Pretrained-ViT/releases/download/0.0.2/B_16_imagenet1k.pth" to /Users/pedrovela/.cache/torch/hub/checkpoints/B_16_imagenet1k.pth
100%|████████████████████████████████████████| 331M/331M [00:08<00:00, 38.7MB/s]


Loaded pretrained weights.


In [7]:
#!pip3 install torchvision

import json
from PIL import Image
import torch
from torchvision import transforms

from functions import general_functions as gf

ruta_imagenes_train = gf.get_data_path('train_images')
img_name = ruta_imagenes_train + '993123.jpeg'

# Load ViT
from pytorch_pretrained_vit import ViT
model = ViT('B_16_imagenet1k', pretrained=True)
model.eval()

# Load image
# NOTE: Assumes an image `img.jpg` exists in the current directory
img = transforms.Compose([
    transforms.Resize((384, 384)), 
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5),
])(Image.open(img_name)).unsqueeze(0)
print(img.shape) # torch.Size([1, 3, 384, 384])

# Classify
with torch.no_grad():
    outputs = model(img).squeeze(0)
print(outputs.shape)  # (1, 1000)

Loaded pretrained weights.
torch.Size([1, 3, 384, 384])
torch.Size([1000])


In [8]:
outputs

tensor([ 4.1168e-02, -3.6647e-01, -8.2826e-02, -5.2003e-03, -6.9432e-01,
         6.3112e-01,  1.1538e-01, -9.5176e-01, -2.8889e-01,  1.1204e-01,
         9.6513e-01, -3.8003e-01, -2.1552e-01, -6.1709e-01,  8.0917e-01,
        -2.5889e-01,  5.4547e-02, -1.1185e+00, -5.8974e-01, -3.2512e-01,
        -2.5198e-01,  5.5554e-01, -2.1421e-02,  3.8290e-01, -2.5087e-02,
         1.9608e+00,  3.3388e+00,  2.1743e+00,  1.2444e+00,  8.5103e-01,
         9.4214e-01,  1.1039e+00,  2.9827e+00, -1.2655e+00,  3.1554e-01,
         9.3914e-01,  8.1178e-01,  4.6449e-01,  2.8589e+00,  3.0353e-01,
         1.3648e+00,  3.1621e+00,  7.6384e-01,  1.6017e+00,  2.9447e+00,
         1.9694e+00,  1.7545e+00,  2.4520e+00, -8.3982e-03,  6.9470e-02,
        -5.9563e-01, -4.6310e-01,  3.2046e+00,  2.7494e+00,  2.3458e+00,
         1.7325e+00,  2.8559e+00,  1.9144e+00,  9.2152e-01,  2.1196e+00,
         1.6230e+00,  9.6469e-01,  1.4489e+00,  1.3848e+00,  1.8677e+00,
         6.0899e-01,  1.9174e+00,  2.1616e+00,  2.7

In [11]:
torch.topk(outputs, k=4).indices.tolist()

[322, 311, 70, 320]

In [12]:
torch.softmax(outputs, -1)[322].item()

0.04724839702248573

El siguiente ejemplo se tomó de [Kaggle](https://www.kaggle.com/code/raufmomin/vision-transformer-vit-from-scratch)

In [5]:
#Importamos las librerias a utlizar

#!pip3 install tensorflow_addons
#!pip3 install --upgrade typing-extensions==4.5
#!pip3 install --upgrade tensorflow==2.13
#!pip3 install 'keras<3.0.0' mediapipe-model-maker

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow_addons as tfa
import glob, random, os, warnings
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

print('TensorFlow Version ' + tf.__version__)

def seed_everything(seed = 0):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed_everything()
warnings.filterwarnings('ignore')

TensorFlow Version 2.13.0


FUNCION DE PARECIDO, PUEDE SER LA FUNCION EUCLIDIANA, COSENO (TE QUEDA AL REVÈS) CUANDO COINCIDE VALE 1 Y CUANDO NO, VALE 0. TENGO LOS EMBEDDINGS. EN UNA CIERTA CAPA DE LA RED TENGO CIERTOS COMPONENTES PRINCIPALES. RED DE REGULARIZACIONES PARA QUE NO SE TE SALTEN EN LAS CAPAS.

PROBLEMA DE CLASIFICACION, MUESTRA DE TRAIN, UNA DE TEST. TENGO UNA IMAGEN DE TEST, LA TRUNCO, SACO LOS RESULTADOS Y OBTENDO SIMILITUDES. SE PUEDE METER EN EL ARQUETIPO A PRIORI. COMO HACER UNA METRICA PARA QUE SE PAREZCA 

TEXTO DESTRUCTURADO, BASE DE DATOS DE GRAFOS (GRAFOS), LO METES RFLIP.

CONTRUIR LOS ARQUETIPOS, NIVEL AL CUAL ESTOY CRTANDO LA RED Y LA METRICA PARA LA COMPARACION. ETIQUETAS: LO QUE ESTOY VIENDO ES QUE ESA MUESTRA A QUE CLUSTER CORRESPONDE. ENTRENAR EL MODELO CON IMAGENES "EQUIS" DE PERROS, ANIMALES, ETC, ETC. PARA TENERLO SUPERVISADO EN EL ENTRENAMIENTO, IR POR FASES Y UNA VEZ QUE HAYA AGARRADO VELOCIDAD ENTONCES SI METERLE LO DE LAS PLANTAS. SEMISUPERVISADO -> IR ALARGANDOLO PARA QUE VAYA APRENDIENDO DE DIFERENTES RAMAS. 

DEEP LEARNING MUY PODEROSO.



## Prueba extra: Solución de Kaggle

Aquí se va a probar una solución proveída por el equipo de Keras dentro del mismo reto de Kaggle. Esto con el fin de poder entender como se puede ir aplicando el modelo y el propósito es que de aquí se van obteniendo vertientes diferentes.

Primero, se probará si es que se puede replicar con mis propios recursos la misma solución para ver si es factible desde mis recursos.

Una vez que tenga la confirmación de que puedo replicar la solución con mis recursos, entonces se investigará que parte(s) es la que puedo editar para tener algo diferente y de ahí ir obteniendo mi proyecto.

Ya que tengo la teoría investigada de mi lado entonces si entraríamos en la fase de experimentación

[Referencia en página de Kaggle](https://www.kaggle.com/code/awsaf49/planttraits2024-kerascv-starter-notebook#🔍-%7C-Loss-&-Metric)

**OTRAS REFERENCIAS:**

[Keras CV Models](https://keras.io/api/keras_cv/models/)

[Train an image classifier from scratch - Keras CV](https://keras.io/guides/keras_cv/classification_with_keras_cv/)

[Learning Rate Schedulers](https://github.com/Mr-TalhaIlyas/Learning-Rate-Schedulers-Packege-Tensorflow-PyTorch-Keras?tab=readme-ov-file)

[SGDR](https://arxiv.org/pdf/1608.03983)

[batch function doc](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#batch)

[StratifiedKFold - Medium](https://medium.com/@literallywords/stratified-k-fold-with-keras-e57c487b1416)

[StratifiedKFold - scikit learn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html)

[StratifiedKFold - Visualization](https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#sphx-glr-auto-examples-model-selection-plot-cv-indices-py)

### Replicación de la solución

En la solución publicada de Kaggle, el modelo que utlizan es **EfficientNev2** (áre ade oportunidad para cambiar de modelo?). 

Partes (pasos) esenciales para la ejecución de este proyecto:

- Designing a data pipeline for a multi-input and multi-output model.
- Creating random augmentation pipeline with KerasCV.
- Loading the data efficiently using tf.data.
- Creating the model using KerasCV presets.
- Training the model.
- Inference and Submission on test data.

In [2]:
#Instalación de las librerías necesarias a utilizar

#!pip3 install keras_cv==0.8.2 --no-deps
#!pip3 install tensorflow==2.15.0 --no-deps
#!pip3 install keras==3.0.4 --no-deps

#Importamos las librerias que vayamos a necesitar

import os
os.environ["KERAS_BACKEND"] = "jax" #Aqui el autor dice que también podemos utilizar tensorflow o torch (ÁREA DE OPORTUNIDAD)

import keras_cv
import keras
from keras import ops
import tensorflow as tf

import cv2
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
import joblib

import matplotlib.pyplot as plt

#Funciones creadas para facilitar algunos procesos
import functions.general_functions as gf

2024-09-17 20:15:53.440906: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
#Imprimimos las versiones de las librerias a utilizar 

print("TensorFlow", tf.__version__)
print("Keras",keras.__version__)
print("KerasCV",keras_cv.__version__)

TensorFlow 2.15.0
Keras 3.0.4
KerasCV 0.8.2


In [4]:
#Iniciamos con la fase de configuración: Diseño de un data pipeline para un modelo con entradas y salidas múltiples

#Configuración de la clase CFG:

class CFG:
    verbose = 1 #Parámetro que configura la manera en que el proceso de entrenamiento es presentado para cada epoch. 0 = silencioso (no muestra nada), 1 = se muestra cada epoch separado por barra, 2 = se muestra solo el número de epoch
    seed = 42 #Semilla que se utiliza para poder reproducir resultados similares en cada ejecución
    preset = "efficientnetv2_b2_imagenet" #Nombre del clasificador pre entrenado, ÁREA DE MEJORA. También se puede utilizar modelos como CSPDarkNet, densenet, EfficientNetV2 (otras vertientes), MiT, MobileNetV3, ResNetV1, YOLO, VitDet
    image_size = [224, 224] #Tamaño de la imagen de entrada del modelo , ÁREA DE MEJORA, investigar que otros tamaños podemos utilizar y como es que cambia los resultados con otros tamaños -> posible efecto que tiene el tamaño de la imagen en los resultados del modelo
    epochs: 12 #Epochs para el entrenamiento -> Número total de iteraciones que tuvo todo tu conjunto de entrenamiento para entrenar el modelo de ML, un "paso" de la data de entrenamiento se toma en cuenta cuando ésta haya pasado tanto por delante como por detrás en todos los procesos del modelo. Incrementando el número de epochs mejora el performance del modelo al permitirle aprender comportamientos más complejos en la data, pero si existen muchos epochs, el modelo se sobre-entrena
    batch_size: 96 #Tamaño del batch -> Un batch es un subconjunto/grupo del conjunto completo de entrenamiento, también se pueden definir como muestras del conjunto de entrenamiento, esto para mejorar el proceso de un conjunto enorme de datos en grupos "pequeños" de datos
    lr_mode = "step" #Tipo de Learning Rate Scheduler y puede ser entre "cos", "step" o "exp". "step" = Step Decay (empieza a deacer el LR después de ciertos epochs mediante un factor lr_decay), "exp" = Exponential Decay (empieza a decaer el LR de manera exponencial), "cos" = Cosine Decay (el LR empieza a decaer mediante un factor de coseno). También hay otros LRS, ÁREA DE MEJORA
    drop_remainder = True #Parámetro lógico para desechar los batches incompletos. Sirve para tener un control sobre los batches en cuanto al tamaño de éstos, si el parámetro es True entonces desechará el último batch si es que éste es de un tamaño menor (batch_size) a los demás, por defualt es FALSE
    num_classes = 6 #Número de clases que vienen en el dataset (las clases que vamos a predecir)
    num_folds = 5 #Número de dobleces (batches) para dividir el conjunto de datos (Prueba y entrenamiento). Se utiliza dentro de la función StratifiedKFold y se refiere a que de los K batches que se crearon de la data, entrenamos el modelo con K-1 batches y probamos con el último batch, esto se repite para todas las permutaciones de batches (al final siempre se hace el entrenamiento dejando un batch diferente fuera cada vez), lo que lo hace estratificado es que también nos fijamos en la distribución relativa de las clases, si una clase aparece más que otra entonces se toma en cuenta para la formación de batches. ÁREA DE MEJORA, podemos utilizar diferentes métodos de fold
    fold = 0 #Número de fold que utilizaremos para la validación de la data. solo sirve para controlar los índices del conjunto de datos y dividirlo en conjunto de prueba y entrenamiento. Aquí definimos que el conjunto para validación será aquel del fold 0, lo demás será para entrenmaiento
    class_names = ['X4_mean', 'X11_mean', 'X18_mean',
                   'X26_mean', 'X50_mean', 'X3112_mean'] #Nombre de las clases del conjunto de datos, son las variables que vamos a predecir
    aux_class_names = list(map(lambda x: x.replace("mean","sd"), class_names)) #Nombre de las clases auxiliares para el entrenamiento de los datos, estas variables vienen también en el conjunto de datos
    num_classes = len(class_names) #Número total de clases que se tiene en el conjunto de entrenamiento
    aux_num_classes = len(aux_class_names) #Número total de clases auxiliares que se tiene en el conjunto de entrenamiento

#Ruta de donde se leerán todos los archivos CSV

ruta_data_csv = gf.get_data_path("csv")

#Ruta de donde se leerán todas las imágenes de entrenamiento

ruta_img_train = gf.get_data_path("train_images")

#Ruta de donde se leerán todas las imágenes de prueba

ruta_img_test = gf.get_data_path("test_images")

In [5]:
#Configuramos el parámetro de seed

keras.utils.set_random_seed(CFG.seed)

In [6]:
#Leemos el archivo de entrenamiento
df = pd.read_csv(f'{ruta_data_csv}/train.csv')
#Creamos una nueva columna con la dirección de cada imagen correspondiente
df['image_path'] = f'{ruta_img_train}/'+df['id'].astype(str)+'.jpeg'
#Sustituimos los NAs en las variables objetivo auxiliares con "-1"
df.loc[:,CFG.aux_class_names] = df.loc[:,CFG.aux_class_names].fillna(-1)
#Mostramos los primeros 2 renglones del conjunto de entrenamiento
display(df.head(2))

#Leemos el conjunto de prueba
test_df = pd.read_csv(f'{ruta_data_csv}/test.csv')
#Creamos una nueva columna con la dirección de cada imagen correspondiente
test_df['image_path'] = f'{ruta_img_train}/'+test_df['id'].astype(str)+'.jpeg'
#Guardamos en una lista el nombre de todas las columnas del conjunto de prueba, estos son las columnas de las características de las plantas
FEATURE_COLS = test_df.columns[1:-1].tolist()
#Mostramos los primeros 2 renglones del conjunto de prueba
display(test_df.head(2))

,id,WORLDCLIM_BIO1_annual_mean_temperature,WORLDCLIM_BIO12_annual_precipitation,WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month,WORLDCLIM_BIO15_precipitation_seasonality,WORLDCLIM_BIO4_temperature_seasonality,WORLDCLIM_BIO7_temperature_annual_range,SOIL_bdod_0.5cm_mean_0.01_deg,SOIL_bdod_100.200cm_mean_0.01_deg,SOIL_bdod_15.30cm_mean_0.01_deg,...,X26_mean,X50_mean,X3112_mean,X4_sd,X11_sd,X18_sd,X26_sd,X50_sd,X3112_sd,image_path
0,192027691,12.235703,374.466675,62.524445,72.256844,773.592041,33.277779,125,149,136,...,1.243779,1.849375,50.216034,0.008921,1.601473,0.025441,0.153608,0.279610,15.045054,/Users/pedrovela/Docs/Datasets - ML/planttrait...
1,195542235,17.270555,90.239998,10.351111,38.220940,859.193298,40.009777,124,144,138,...,0.642940,1.353468,574.098472,0.003102,0.258078,0.000866,0.034630,0.010165,11.004477,/Users/pedrovela/Docs/Datasets - ML/planttrait...


,id,WORLDCLIM_BIO1_annual_mean_temperature,WORLDCLIM_BIO12_annual_precipitation,WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month,WORLDCLIM_BIO15_precipitation_seasonality,WORLDCLIM_BIO4_temperature_seasonality,WORLDCLIM_BIO7_temperature_annual_range,SOIL_bdod_0.5cm_mean_0.01_deg,SOIL_bdod_100.200cm_mean_0.01_deg,SOIL_bdod_15.30cm_mean_0.01_deg,...,VOD_X_1997_2018_multiyear_mean_m04,VOD_X_1997_2018_multiyear_mean_m05,VOD_X_1997_2018_multiyear_mean_m06,VOD_X_1997_2018_multiyear_mean_m07,VOD_X_1997_2018_multiyear_mean_m08,VOD_X_1997_2018_multiyear_mean_m09,VOD_X_1997_2018_multiyear_mean_m10,VOD_X_1997_2018_multiyear_mean_m11,VOD_X_1997_2018_multiyear_mean_m12,image_path
0,195066138,10.55810,961.500000,31.586735,13.728325,648.038208,25.351532,127,152,137,...,0.469694,0.455849,0.528211,0.555653,0.549882,0.542905,0.517507,0.462724,0.427107,/Users/pedrovela/Docs/Datasets - ML/planttrait...
1,195524180,7.00287,1120.025513,23.000000,7.258863,973.889404,39.135712,106,167,127,...,0.428838,0.456266,0.470074,0.468038,0.475943,0.483206,0.477197,0.432732,0.423728,/Users/pedrovela/Docs/Datasets - ML/planttrait...


In [9]:
#Crearemos un data loader el cual podrá leer imágenes y archivos CSV al mismo tiempo, también maneja etiquetas para tareas principales y auxiliares
#Este data loader también aplicará modificaciones (augmentations) a las imágenes, tales como flip, rotation, brightness, etc. Todas estas modificaciones se aplicará solo a un subconjunto de imágenes. lo cual acelera el entrenamiento del modelo y reduce el cuello de botella del CPU

#Función para contruir el modificador de las imágenes
def build_augmenter():
    #Definimos la modificaciones que se le harán a las imágenes
    aug_layers = [
        #Brillo de la imagen que será ajustado de manera aleatoria durante el proceso de entrenamiento del modelo [https://keras.io/api/layers/preprocessing_layers/image_augmentation/random_brightness/]
        keras_cv.layers.RandomBrightness(factor = 0.1, value_range = (0, 1)),
        #Contraste de la imagen que será ajustado de manera aleatoria durante el proceso de entrenamiento del modelo [https://keras.io/api/layers/preprocessing_layers/image_augmentation/random_contrast/]
        keras_cv.layers.RandomContrast(factor = 0.1, value_range = (0, 1)),
        #Saturación de la imagen que será ajustado de manera aleatoria para reducir o aumentar la saturación de la imagen de input [https://keras.io/api/keras_cv/layers/augmentation/random_saturation/]
        keras_cv.layers.RandomSaturation(factor = (0.45, 0.55)),
        #Color a o matiz de la imagen que será ajustado de manera aleatoria [https://keras.io/api/keras_cv/layers/augmentation/random_hue/]
        keras_cv.layers.RandomHue(factor = 0.1, value_range = (0, 1)),
        #Recorte aleatorio de rectángulos de las imágenes para después rellenarlas. Aquí se pueden hacer rellenos gausianos [https://keras.io/api/keras_cv/layers/augmentation/random_cutout/]
        keras_cv.layers.RandomCutout(height_factor = (0.06, 0.15), width_factor = (0.06, 0.15)),
        #Voltea las imágenes de manera aleatoria durante el proceso de entrenamiento [https://keras.io/api/layers/preprocessing_layers/image_augmentation/random_flip/]
        keras_cv.layers.RandomFlip(mode = "horizontal_and_vertical"),
        #Hace zoom (de lejos o cerca) a las imágenes de manera aleatoria durante el proceso de entrenamiento del modelo [https://keras.io/api/layers/preprocessing_layers/image_augmentation/random_zoom/]
        keras_cv.layers.RandomZoom(height_factor = (0.05, 0.15)),
        #Rotación aleatoria de las imágenes durante el proceso de entrenamiento del modelo [https://keras.io/api/layers/preprocessing_layers/image_augmentation/random_rotation/]
        keras_cv.layers.RandomRotation(factor = (0.01, 0.05))
    ]

    #Aplicamos las modificaciones a muestras aleatorias 
    aug_layers = [keras_cv.layers.RandomApply(x, rate = 0.5) for x in aug_layers]

    #Construimos la capa de modificaciones
    augmenter = keras_cv.layers.Augmenter(aug_layers)

    #Aplicamos las modificaciones
    def augment(inp, label):
        images = inp["images"]
        aug_data = {"images": images}
        aug_data = augmenter(aug_data)
        inp["images"] = aug_data["images"]
        return inp, label
    return augment

    return augmenter

In [10]:
x1 = build_augmenter()
x1

AttributeError: 'Augmenter' object has no attribute 'name'